In [106]:
from selenium import webdriver
# from selenium.webdriver.firefox.options import Options
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import langchain_ollama

from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
import time
from datetime import datetime


MY_NAME = "Vikas Reddy"

#### Creating a selenium based chrome driver for actions

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager  # optional, but handy

options = Options()
# point at the running Chrome on 9222
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

# if your Chrome binary isn’t on PATH, also do:
# "C:\Program Files\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9222 --user-data-dir="C:\ChromeDebug" --no-first-run --no-default-browser-check

print("Using Chrome binary:", options.binary_location)
# pick a driver that matches your Chrome version:
service = Service(ChromeDriverManager().install())

print("Using ChromeDriver binary:", service.path)
# this will attach instead of launching a new window
driver = webdriver.Chrome(service=service, options=options)

print("Attached to Chrome, current URL:", driver.current_url)
driver.get("https://www.linkedin.com/search/results/people")



Using Chrome binary: 
Using ChromeDriver binary: C:\Users\VIKAS\.wdm\drivers\chromedriver\win64\136.0.7103.92\chromedriver-win32/chromedriver.exe
Attached to Chrome, current URL: https://www.linkedin.com/messaging/thread/2-YWY4ZjY3MWEtOWExNy00NTYwLWFkMTctOWY2YTBiMmE4ZTU5XzAxMg==/


In [19]:
## click on the message bar

def click_on_message():
    message_bar = driver.find_element(By.XPATH, "//span[normalize-space(.)='Messaging']")
    message_bar.click()

click_on_message()  

In [20]:
# scrolls upwards for the top of the page for the user

def wheel_scroll_up(amount=-600, repeats=5, pause=0.10):
    """
    Spin the mouse wheel upward inside the message list.

    amount   – vertical pixels per wheel tick  (negative = up, positive = down)
    repeats  – how many wheel ticks to send
    pause    – delay between ticks so LinkedIn can lazy-load older messages
    """
    # 1. locate the scrollable <div>
    scroll_el = driver.find_element(
        By.CSS_SELECTOR, ".msg-s-message-list.full-width.scrollable"
    )

    # 2. define where the wheel should originate (center of the element)
    origin = ScrollOrigin.from_element(scroll_el)

    # 3. build the wheel actions
    actions = ActionChains(driver)
    for _ in range(repeats):
        actions.scroll_from_origin(origin, 0, amount)
        actions.pause(pause)        # give time for lazy-loading

    actions.perform()

# example: yank the scrollbar up ~3 000 px in 5 smooth “ticks”



In [21]:
## this function is check if messages time into a particular format

def is_valid_datetime_format(date_string):
    date_string = date_string.strip()  # Remove leading/trailing whitespace
    try:
        datetime.strptime(date_string, "%b %d, %Y %I:%M %p")
        return True
    except ValueError:
        return False

# sanity
# is_valid_datetime_format(" Apr 5, 2024 11:31 PM")  # True

In [22]:
# this function is to get the time of the message

def time_formatter(raw):# 1) Strip leading/trailing whitespace
    raw_clean = raw.strip()

    # 2) Parse into a datetime object (24-hour output by default)
    dt = datetime.strptime(raw_clean, "%b %d, %Y %I:%M %p")
    return dt

In [ ]:
from datetime import timedelta

# Initialize a list to store messages in the desired format
messages_data = []

def retrieve_profile_messages(message_id):
    # go to the top of the conversation
    wheel_scroll_up(amount=-600, repeats=5)

    event_elements = driver.find_elements(By.XPATH, "//li[contains(@class, 'msg-s-message-list__event clearfix')]")
    flag = True
    a_tag_with_open = driver.find_element(By.XPATH, "//a[contains(@title, 'Open')]")
    # print(a_tag_with_open.get_attribute("title"))

    name = a_tag_with_open.get_attribute("title")
    name = name.replace("’s profile", "").replace("Open ", "")  # Handle curly apostrophe as well
    print("name of the user:", name)

    prev_message = None
    p_tag_text = ""
    for indx, event in enumerate(event_elements):
        
        
        # print("Event Index:", indx, event.get_attribute("class"))

        # 1) Find the <time> node inside this <li>
        time_elems = event.find_elements(By.TAG_NAME, "time")
    
        div_with_title = driver.find_element(By.XPATH, ".//div[@title]")
        print("title :: ", div_with_title.get_attribute("title"))

        time_value = ""
        time_count = 0
        for i in time_elems:
            # print(i.text)
            try:
                if datetime.strptime(i.text, "%b %d"):
                    time_value = i.text + f", {datetime.now().year}"
            except :
                time_value += f" {i.text}"
            time_count += 1
        
        if time_count <= 1:
            time_value = f" {prev.split()[0]} {prev.split()[1]} {prev.split()[2]}" + time_value
        
        prev = time_value
        print("Time Value:", time_value)

        # Convert "THURSDAY 9:13 PM" or "TODAY 9:13 PM" to "%b %d, %Y %I:%M %p" format
        if time_value.split()[0].upper() in ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY", "TODAY"]:
            day_of_week_or_today = time_value.split()[0].upper()
            time_part = time_value.split()[1] + " " + time_value.split()[2]
            today = datetime.now()
            
            if day_of_week_or_today == "TODAY":
                recent_date = today
            else:
                days_ago = (today.weekday() - ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"].index(day_of_week_or_today)) % 7
                recent_date = today - timedelta(days=days_ago)
            
            time_value = recent_date.strftime("%b %d, %Y") + " " + time_part

        if flag == True:
            a_tag = event.find_element(By.XPATH, ".//a[img]")
            img_tag = a_tag.find_element(By.TAG_NAME, "img")
            # print(a_tag.get_attribute("href"))
            # print(img_tag.get_attribute("src"))
            flag = False
            title = img_tag.get_attribute("title")
            # print("Image Title:-----------------", title)
        
        p_tag = event.find_element(By.TAG_NAME, "p") if event.find_elements(By.TAG_NAME, "p") else None
        
        p_tag_text = p_tag.text if p_tag else ""
        print(f"Message: {p_tag_text}", time_value, is_valid_datetime_format(time_value))
    
        if is_valid_datetime_format(time_value) == True:
            
            # print("valid datetime format:", time_value)
            
            if prev_message!= None:
                messages_data.append(prev_message)
                # print("Added to messages_data", prev_message["message_text"])
            prev_message = {
            "message_id": f"messg_{message_id}",
            "conversation_id": f"conv_{indx}",
            "platform": "LinkedIn",
            "connection_name": name,
            "message_sender": title if indx%2==0 else name,
            "message_date": time_formatter(time_value),
            "message_text": p_tag_text,
            "connection_title": div_with_title.get_attribute("title")
            }
            
            
        else:
            # print(time_value)
            time_value = prev_message["message_date"]
            prev_message["message_text"] += f"\n{p_tag_text}"
        print("--------------------------------------------")
        # Store the message details in the desired format
    print("Final Message:", p_tag_text)
    messages_data.append({"message_id": f"messg_{message_id}",
            "conversation_id": f"conv_{indx}",
            "platform": "LinkedIn",
            "connection_name": name,
            "message_sender": title if indx%2==0 else name,
            "message_date": time_formatter(time_value),
            "message_text": p_tag_text,
            "connection_title": div_with_title.get_attribute("title")
            })
    print(p_tag_text)
    print("----------------------------------------------------------------------------------")
        


In [137]:
def get_time(event, prev):
    # 1) Find the <time> node inside this <li>
    time_elems = event.find_elements(By.TAG_NAME, "time")

    time_value = ""
    time_count = 0

    for i in time_elems:
        print(i.text, "i.text")
        try:
            if datetime.strptime(i.text, "%b %d"):
                time_value = i.text + f", {datetime.now().year}"
        except :
            time_value += f" {i.text}"
        time_count += 1
    

    if time_count <= 1:
        print("here before", time_value, prev)
        time_value = f" {prev.split()[0]} {prev.split()[1]} {prev.split()[2]}" + time_value
        print("here after", time_value, prev)
    
    prev = time_value
    
    # Convert "THURSDAY 9:13 PM" or "TODAY 9:13 PM" to "%b %d, %Y %I:%M %p" format
    if time_value.split()[0].upper() in ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY", "TODAY"]:
        day_of_week_or_today = time_value.split()[0].upper()
        time_part = time_value.split()[1] + " " + time_value.split()[2]
        today = datetime.now()
        
        if day_of_week_or_today == "TODAY":
            recent_date = today
        else:
            days_ago = (today.weekday() - ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"].index(day_of_week_or_today)) % 7
            recent_date = today - timedelta(days=days_ago)
        
        time_value = recent_date.strftime("%b %d, %Y") + " " + time_part
    
    return time_value, prev


def get_title(event):
    
    a_tag = event.find_element(By.XPATH, ".//a[img]")
    img_tag = a_tag.find_element(By.TAG_NAME, "img")
    title = img_tag.get_attribute("title")
    
    return title


def get_name_of_the_connection():

    a_tag_with_open = driver.find_element(By.XPATH, "//a[contains(@title, 'Open')]")
    name = a_tag_with_open.get_attribute("title")
    name = name.replace("’s profile", "").replace("Open ", "")  # Handle curly apostrophe as well
    print("name of the user:", name)
    return name


In [138]:
from datetime import timedelta

# Initialize a list to store messages in the desired format
messages_data = []

def retrieve_profile_messages(message_id):

    # go to the top of the conversation
    wheel_scroll_up(amount=-600, repeats=5)

    event_elements = driver.find_elements(By.XPATH, "//li[contains(@class, 'msg-s-message-list__event clearfix')]")
    
    name = get_name_of_the_connection()
    
    prev_message = None
    p_tag_text = ""
    prev = ""
    prev_time = ""


    for indx, event in enumerate(event_elements):
        
        # 1. get the title of the user. 
        div_with_title = driver.find_element(By.XPATH, ".//div[@title]")
        print("title :: ", div_with_title.get_attribute("title"))
        
        # 2. get time_value and prev.
        time_value, prev = get_time(event, prev)

        # 3. get the text values from the table. 
        p_tag = event.find_element(By.TAG_NAME, "p") if event.find_elements(By.TAG_NAME, "p") else None
        p_tag_text = p_tag.text if p_tag else ""

        # 4. get the name of the person sent the message
        try:
            span_elem = event.find_element(By.XPATH, ".//span[contains(@class, 'msg-s-event-listitem--group-a11y-heading')]")
            driver.execute_script("arguments[0].scrollIntoView(true);", span_elem)  # Scroll into view if necessary
            message_sender = driver.execute_script("return arguments[0].innerText;", span_elem)  # Use JavaScript to retrieve innerText
            print("message_sender:", message_sender)
        except Exception as e:
            print("message_sender", message_sender)

        print(time_value, prev, is_valid_datetime_format(time_value))
        if is_valid_datetime_format(time_value):
            msg_time = time_value
            prev_time = time_value
        else:
            msg_time = prev_time
        message_sender = message_sender.split(" sent ")[0]

        prev_message = {
        "message_id": f"messg_{message_id}",
        "conversation_id": f"conv_{indx}",
        "platform": "LinkedIn",
        "connection_name": name,
        "message_sender": message_sender,
        "message_date": time_formatter(msg_time),
        "message_text": p_tag_text,
        "connection_title": div_with_title.get_attribute("title")
        }
            
       
        messages_data.append(prev_message)
        print('---------------------------------------------------')
    print(p_tag_text)
    print("----------------------------------------------------------------------------------")
        


In [140]:
# retrieve_profile_messages(1)
messages_data

[{'message_id': 'messg_1',
  'conversation_id': 'conv_0',
  'platform': 'LinkedIn',
  'connection_name': 'Namruth Thimmapuram',
  'message_sender': 'Vikas Reddy Venkannagari',
  'message_date': datetime.datetime(2024, 11, 4, 20, 28),
  'message_text': "Hi Namruth Goud,\n\nIt's been some time since we connected. We both studied at University of Maryland Baltimore County. How have you been?",
  'connection_title': "Master's in Data Science @UMBC"},
 {'message_id': 'messg_1',
  'conversation_id': 'conv_1',
  'platform': 'LinkedIn',
  'connection_name': 'Namruth Thimmapuram',
  'message_sender': 'Namruth Thimmapuram',
  'message_date': datetime.datetime(2024, 11, 10, 23, 25),
  'message_text': 'Hi, Vikas Reddy',
  'connection_title': "Master's in Data Science @UMBC"},
 {'message_id': 'messg_1',
  'conversation_id': 'conv_2',
  'platform': 'LinkedIn',
  'connection_name': 'Namruth Thimmapuram',
  'message_sender': 'Namruth Thimmapuram',
  'message_date': datetime.datetime(2024, 11, 10, 23, 

In [ ]:
from datetime import timedelta

# Initialize a list to store messages in the desired format
messages_data = []

def retrieve_messages(message_id):
    # go to the top of the conversation
    wheel_scroll_up(amount=-600, repeats=5)

    event_elements = driver.find_elements(By.XPATH, "//li[contains(@class, 'msg-s-message-list__event clearfix')]")

    a_tag_with_open = driver.find_element(By.XPATH, "//a[contains(@title, 'Open')]")

    name = a_tag_with_open.get_attribute("title")
    name = name.replace("’s profile", "").replace("Open ", "")  # Handle curly apostrophe as well
    print("name of the user:", name)

    prev_message = None
    p_tag_text = ""
    for indx, event in enumerate(event_elements):
        
        p_tag = event.find_element(By.TAG_NAME, "p") if event.find_elements(By.TAG_NAME, "p") else None

        p_tag_text = p_tag.text if p_tag else ""
        print(f"Message: {p_tag_text}")

        # Locate the first span element within the event element
        try:
            span_elem = event.find_element(By.XPATH, ".//span[contains(@class, 'msg-s-event-listitem--group-a11y-heading')]")
            driver.execute_script("arguments[0].scrollIntoView(true);", span_elem)  # Scroll into view if necessary
            span_text = driver.execute_script("return arguments[0].innerText;", span_elem)  # Use JavaScript to retrieve innerText
            print("Span text:", span_text)
        except Exception as e:
            print("Error retrieving span text:", span_text)
        # # Locate the span within the div
        # span_with_a_tag = event.find_element(By.XPATH, ".//div[contains(@class, 'msg-s-message-group__meta')]//span/a/span")
        # span_text = span_with_a_tag.text
        # print("Span text inside a tag:", span_text)
 
    print("----------------------------------------------------------------------------------")
        


In [83]:
retrieve_messages(1)

name of the user: Namruth Thimmapuram
Message: Hi Namruth Goud,

It's been some time since we connected. We both studied at University of Maryland Baltimore County. How have you been?
Span text: Vikas Reddy Venkannagari sent the following message at 8:28 PM
Message: Hi, Vikas Reddy
Span text: Namruth Thimmapuram sent the following messages at 11:25 PM
Message: I'm good
Error retrieving span text: Namruth Thimmapuram sent the following messages at 11:25 PM
Message: How about you?
Error retrieving span text: Namruth Thimmapuram sent the following messages at 11:25 PM
Message: I'm good, too
Span text: Vikas Reddy Venkannagari sent the following messages at 8:37 PM
Message: profile pic bagunda ?
Error retrieving span text: Vikas Reddy Venkannagari sent the following messages at 8:37 PM
Message: baane extralu
Error retrieving span text: Vikas Reddy Venkannagari sent the following messages at 8:37 PM
Message: Yeah
Span text: Namruth Thimmapuram sent the following message at 8:37 AM
Message: 

In [ ]:

    # Do something with the title if needed

Software Engineering MTS @ Salesforce | CS Grad - Stony Brook University | Ex-TikTok, Oracle


In [20]:
# .nhnhnhjjnnnnnnnnn.,llk,;lkmn \
# is_valid_datetime_format(" JAN 28, 2024 11:31 PM")  # True
messages_data

[{'message_id': 'messg_1',
  'conversation_id': 'conv_0',
  'platform': 'LinkedIn',
  'connection_name': 'Sri Teja Vure',
  'sender_username': 'Vikas Reddy Venkannagari',
  'message_date': datetime.datetime(2025, 4, 22, 13, 24),
  'message_text': 'Hey Sri Teja, I’m Vikas, a current grad student at UMBC, applying for Software Engineer role at Salesforce. I’d love your take—would you be open to a referral? I bring over 1.5 years of relevant experience and am happy to share my resume. Thanks!',
  'connection_title': 'Software Engineering MTS @ Salesforce | CS Grad - Stony Brook University | Ex-TikTok, Oracle'},
 {'message_id': 'messg_1',
  'conversation_id': 'conv_1',
  'platform': 'LinkedIn',
  'connection_name': 'Sri Teja Vure',
  'sender_username': 'Sri Teja Vure',
  'message_date': datetime.datetime(2025, 4, 23, 16, 38),
  'message_text': 'Hi, sure, which job id are you looking for?',
  'connection_title': 'Software Engineering MTS @ Salesforce | CS Grad - Stony Brook University | Ex-

In [49]:
# clicks on the message bar
# click_on_message()
import time
# goes through each profile in the profiles bar
def retrieve_messages():
    index = 0

    
       
    conversation_list = driver.find_element(By.XPATH, "//ul[@aria-label='Conversation List']")

    for item in conversation_list.find_elements(By.TAG_NAME, "li"):
        if index <= 79:
            index += 1
            continue
        try:
            item.click()
            time.sleep(2)
            retrieve_profile_messages(index)
            index += 1
            time.sleep(random.randint(4, 6))
        except Exception as e:
            print("Error:", e)
            continue
        time.sleep(random.randint(4, 6))
        print(messages_data[-1])

        

In [50]:
messages_data = []

retrieve_messages()

Harsh Dankhara
Data Analyst / Risk Analyst @ BlackRock
APR 13
10:43 AM
Time Value: APR 13, 2025 10:43 AM
Image Title: Vikas Reddy Venkannagari
Message: Hey Harsh, I’m Vikas, a current grad student at UMBC, applying for Data Analyst role at BlackRock. I’d love your take—would you be open to a referral? I bring over 1.5 years of relevant experience and am happy to share my resume. Thanks! APR 13, 2025 10:43 AM True
valid datetime format: APR 13, 2025 10:43 AM
--------------------------------------------
Data Analyst / Risk Analyst @ BlackRock
10:44 AM
Time Value:  APR 13, 2025 10:44 AM
Message: Hi Vikas 
Sorry but I don’t have access to referral portal   APR 13, 2025 10:44 AM True
valid datetime format:  APR 13, 2025 10:44 AM
Added to messages_data Hey Harsh, I’m Vikas, a current grad student at UMBC, applying for Data Analyst role at BlackRock. I’d love your take—would you be open to a referral? I bring over 1.5 years of relevant experience and am happy to share my resume. Thanks!
-----

In [52]:
messages_data

[{'message_id': 'messg_80',
  'conversation_id': 'conv_0',
  'platform': 'LinkedIn',
  'connection_name': 'Harsh Dankhara',
  'message_sender': 'Vikas Reddy Venkannagari',
  'message_date': datetime.datetime(2025, 4, 13, 10, 43),
  'message_text': 'Hey Harsh, I’m Vikas, a current grad student at UMBC, applying for Data Analyst role at BlackRock. I’d love your take—would you be open to a referral? I bring over 1.5 years of relevant experience and am happy to share my resume. Thanks!',
  'connection_title': 'Data Analyst / Risk Analyst @ BlackRock'},
 {'message_id': 'messg_80',
  'conversation_id': 'conv_1',
  'platform': 'LinkedIn',
  'connection_name': 'Harsh Dankhara',
  'message_sender': 'Harsh Dankhara',
  'message_date': datetime.datetime(2025, 4, 13, 10, 44),
  'message_text': 'Hi Vikas \nSorry but I don’t have access to referral portal ',
  'connection_title': 'Data Analyst / Risk Analyst @ BlackRock'},
 {'message_id': 'messg_80',
  'conversation_id': 'conv_2',
  'platform': 'Lin

In [51]:
import json

# Convert the list of dictionaries to a JSON file
with open("messages_data4.json", "w") as json_file:
    json.dump(messages_data, json_file, indent=4, default=str)  # Use default=str to handle datetime serialization


In [260]:
span_elem = driver.find_element(
    By.XPATH,
    "//span[contains(@class,'msg-s-event-listitem--group-a11y-heading') and contains(@class,'visually-hidden')]"
)
print(span_elem.text)
